![tf](images/tficon-small.png)

# Tutorial

We show the basic functions of the
[TF-API](https://dans-labs.github.io/text-fabric/Api/General/).

See this [readme](https://github.com/pthu/pilot) for more information about
the corpus and this work.

Start with a cell that enables live reloading: if you change imported modules,
the changes will be effective without restarting the notebook.

In [1]:
%load_ext autoreload
%autoreload 2

Import Text-Fabric

In [2]:
from tf.fabric import Fabric

Put in a few sign posts to directories on your computer.

In [3]:
REPO = '~/github/pthu/pilot'

Start Text-Fabric and let it look in the relevant data directories.

In [4]:
TF = Fabric(locations=REPO, modules=['tf'])

This is Text-Fabric 5.5.11
Api reference : https://dans-labs.github.io/text-fabric/Api/General/
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

14 features found and 0 ignored


Load all features that TF has found.

In [16]:
api = TF.load('', silent=True)
allFeatures = TF.explore(silent=True, show=True)
loadableFeatures = allFeatures['nodes'] + allFeatures['edges']
print('Features:')
print(', '.join(loadableFeatures))
TF.load(loadableFeatures, add=True, silent=True)
api.makeAvailableIn(globals())

Features:
book, book@en, chapter, main, orig, otype, page, para, plain, post, pre, verse, oslots


# Feature analysis

Let's explore the values of some features and show their frequencies, especially the potentially
troublesome features that contain non-letter material at the start and end of words.

In [6]:
for (val, freq) in F.pre.freqList():
    print(f'{val:<10} {freq:>4}x')

[            74x
—             7x
?̔            4x
⟦             1x


In [7]:
for (val, freq) in F.post.freqList():
    print(f'{val:<10} {freq:>4}x')

           13546x
,           816x
.           679x
·           295x
;           170x
]            68x
].            3x
̓.            3x
.]            2x
?̔            2x
*             1x
,             1x
,—            1x
.⟧            1x
?̓            1x
[,            1x
],            1x
]·            1x
·?̔           1x
·̓            1x


We can show frequency lists of more interesting features as well:

In [8]:
for (val, freq) in F.plain.freqList()[0:20]:
    print(f'{val:<10} {freq:>4}x')

ΚΑΙ         827x
Ο           583x
ΟΤΙ         274x
ΤΟΥ         242x
ΤΟΝ         240x
ΕΝ          233x
ΔΕ          209x
ΕΙΣ         206x
ΟΥΝ         198x
ΙΗΣΟΥΣ      196x
ΑΥΤΟΝ       175x
ΑΥΤΩ        173x
ΑΥΤΟΥ       171x
ΕΣΤΙΝ       166x
ΟΥΚ         151x
ΤΟ          147x
ΟΙ          146x
Η           145x
ΙΝΑ         145x
ΤΗΝ         143x


In [9]:
for (val, freq) in F.main.freqList()[0:20]:
    print(f'{val:<10} {freq:>4}x')

καὶ         803x
ὁ           521x
ὅτι         270x
τοῦ         242x
τὸν         238x
ἐν          216x
δὲ          199x
οὖν         197x
Ἰησοῦς      195x
εἰς         186x
αὐτοῦ       171x
αὐτῷ        171x
τὸ          147x
ἵνα         145x
τὴν         141x
οἱ          136x
οὐκ         134x
ἐκ          133x
λέγει       119x
ἡ           116x


# Navigation

Let's go to John 3:16.

Note that we pass the section as a tuple `(book, chapter, verse)`.

In [10]:
j316 = T.nodeFromSection(('John', 3, 16))
print(j316)

15709


Let's go from this verse to the surrounding chapter and book.

Note that `L.u()` gives us a tuple of nodes. Mostly, going *up* yields a tuple of only one
node.

Yet,in general,
it is conceivable that a node has multiple nodes that contain it, even of the same type.

Hence, to get to the single parent, we have to put in the `[0]`.

In [11]:
j3 = L.u(j316, otype='chapter')[0]
j = L.u(j316, otype='book')[0]

As a check, let's print the name of the book and the number of the chapter:

In [12]:
print(f'The book is {F.book.v(j)}')
print(f'The chapter is {F.chapter.v(j3)}')
print(f'The verse is {F.verse.v(j316)}')

The book is John
The chapter is 3
The verse is 16


# Text formats

Which text formats do we have in our datasets?

In [13]:
T.formats

{'text-orig-full', 'text-orig-main', 'text-orig-plain'}

Let's print some text. John 3:16 comes to mind. We do it in all formats.
We also print the verse label. We only need the node of the verse to get started.

We wrap the whole thing in a function, that takes a section tuple as input.
Optionally you can specify in which formats you want the text printed, but if you don't,
you get all formats.

In [27]:
def printVerse(section, fmts=None, hideFmt=False):
    node = T.nodeFromSection(section)
    print('{} {}:{}'.format(*section))
    words = L.d(node, otype='word')
    for fmt in T.formats if fmts is None else fmts:
        if not hideFmt:
            print(f'--- {fmt} ---')
        print(T.text(words, fmt=fmt))
        print('')

In [28]:
printVerse(('John', 3, 16))

John 3:16
--- text-orig-main ---
Οὕτως γὰρ ἠγάπησεν ὁ θεὸς τὸν κόσμον ὥστε τὸν υἱὸν τὸν μονογενῆ ἔδωκεν ἵνα πᾶς ὁ πιστεύων εἰς αὐτὸν μὴ ἀπόληται ἀλλὰ ἔχῃ ζωὴν αἰώνιον 

--- text-orig-plain ---
ΟΥΤΩΣ ΓΑΡ ΗΓΑΠΗΣΕΝ Ο ΘΕΟΣ ΤΟΝ ΚΟΣΜΟΝ ΩΣΤΕ ΤΟΝ ΥΙΟΝ ΤΟΝ ΜΟΝΟΓΕΝΗ ΕΔΩΚΕΝ ΙΝΑ ΠΑΣ Ο ΠΙΣΤΕΥΩΝ ΕΙΣ ΑΥΤΟΝ ΜΗ ΑΠΟΛΗΤΑΙ ΑΛΛΑ ΕΧΗ ΖΩΗΝ ΑΙΩΝΙΟΝ 

--- text-orig-full ---
Οὕτως  γὰρ  ἠγάπησεν  ὁ  θεὸς  τὸν  κόσμον  ὥστε  τὸν  υἱὸν  τὸν  μονογενῆ  ἔδωκεν,  ἵνα  πᾶς  ὁ  πιστεύων  εἰς  αὐτὸν  μὴ  ἀπόληται  ἀλλὰ  ἔχῃ  ζωὴν  αἰώνιον.  



Print the whole chapter in text-orig-main format.

In [29]:
chapter = T.nodeFromSection(('John', 3))
for verse in L.d(chapter, otype='verse'):
    section = T.sectionFromNode(verse)
    printVerse(section, fmts={'text-orig-main'}, hideFmt=True)

John 3:1
Ἦν δὲ ἄνθρωπος ἐκ τῶν Φαρισαίων Νικόδημος ὄνομα αὐτῷ ἄρχων τῶν Ἰουδαίων 

John 3:2
οὗτος ἦλθεν πρὸς αὐτὸν νυκτὸς καὶ εἶπεν αὐτῷ Ῥαββεί οἴδαμεν ὅτι ἀπὸ θεοῦ ἐλήλυθας διδάσκαλος οὐδεὶς γὰρ δύναται ταῦτα τὰ σημεῖα ποιεῖν ἃ σὺ ποιεῖς ἐὰν μὴ ᾖ ὁ θεὸς μετʼ αὐτοῦ 

John 3:3
ἀπεκρίθη Ἰησοῦς καὶ εἶπεν αὐτῷ Ἀμὴν ἀμὴν λέγω σοι ἐὰν μή τις γεννηθῇ ἄνωθεν οὐ δύναται ἰδεῖν τὴν βασιλείαν τοῦ θεοῦ 

John 3:4
λέγει πρὸς αὐτὸν ὁ Νικόδημος Πῶς δύναται ἄνθρωπος γεννηθῆναι γέρων ὤν μὴ δύναται εἰς τὴν κοιλίαν τῆς μητρὸς αὐτοῦ δεύτερον εἰσελθεῖν καὶ γεννηθῆναι 

John 3:5
ἀπεκρίθη ὁ Ἰησοῦς Ἀμὴν ἀμὴν λέγω σοι ἐὰν μή τις γεννηθῇ ἐξ ὕδατος καὶ πνεύματος οὐ δύναται εἰσελθεῖν εἰς τὴν βασιλείαν τοῦ θεοῦ 

John 3:6
τὸ γεγεννημένον ἐκ τῆς σαρκὸς σάρξ ἐστιν καὶ τὸ γεγεννημένον ἐκ τοῦ πνεύματος πνεῦμά ἐστιν 

John 3:7
μὴ θαυμάσῃς ὅτι εἶπόν σοι Δεῖ ὑμᾶς γεννηθῆναι ἄνωθεν 

John 3:8
τὸ πνεῦμα ὅπου θέλει πνεῖ καὶ τὴν φωνὴν αὐτοῦ ἀκούεις ἀλλʼ οὐκ οἶδας πόθεν ἔρχεται καὶ ποῦ ὑπάγει οὕτως ἐστὶν πᾶς ὁ γεγεννημένος ἐκ 